<div style="background-color:#000047; padding: 30px; border-radius: 10px; color: white; text-align: center;">
    <img src='Figures/alinco.png' style="height: 100px; margin-bottom: 10px;"/>
    <h1>Redes Neuronales Multiclase</h1>
</div>

   
Los clasificadores no solo trabajan sobre dos clases, es muy común encontrarnos con problemas con más de dos. Para adaptar una red neuronal a un problema de $n$ clases basta con colocar en la capa de salida $n$ neuronas. 

<img src="Figures/multiclass.svg" width="60%">

### Clasificación de Plantas Iris
Vamos a estudiar esto haciendo uso de un *dataset* muy conocido utilizado para clasificar el género de plantas iris. La ﬂor de iris se presenta en tres especies distintas denominadas: 

- setosa
- versicolor
- virgínica

Pueden identiﬁcarse atendiendo a los anchos y altos de sus pétalos y sépalos. Este conjunto de datos contiene 150 muestras de estas ﬂores, recogiendo 50 muestras de cada especie. 


**Las cuatro primeras columnas corresponden al alto y ancho del sépalo y al alto y ancho del pétalo. La quinta columna indica la especie a la que pertenece:** 

- 0 → setosa, 
- 1 → versicolor
- 2 → virgínica. 


<img src="Figures/iris.png" width="25%">


Vemos que la quinta columna contiene los valores 0,1 y 2. Son las etiquetas correspondientes a las tres clases. Decíamos que la forma de adaptar ahora una red para clasificar tres clases es poner en la capa de salida tres neuronas. Recordemos que la función de activación que estamos usando es la sigmoide, cuyo rango es $(0,1)$. ¿Cómo hacemos encajar esto para que podamos definir una función de error sobre la cual podamos descender por el gradiente hasta un mínimo?


In [2]:
import pandas as pd

In [3]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target
data_iris = pd.DataFrame(data=X, columns = iris.feature_names)
data_iris['especie'] = y

In [4]:
data_iris['out'] = data_iris.iloc[:,4].astype('category').cat.codes

data_iris['out'].unique()

array([0, 1, 2], dtype=int8)


### Codificación one-hot

La solución está en utilizar la codificación **one-hot!!**. 

Esta codificación hace lo siguiente, se toma un vector de una dimensión igual al número de clases que haya, en este caso, 3. Todas las componentes del vector estarán a 0, excepto una que tendrá el valor 1 y que corresponderá con la clase que defina. Es decir:

- 0 →[1,0,0], 
- 1 →[0,1,0] , 
- 2 →[0,0,1]. 




In [7]:
import numpy as np
def one_hot(x, n):
    if type(x)==list:
        x = np.array(x)
    x = x.flatten()
    o_h = np.zeros((len(x), n))
    o_h[np.arange(len(x)), x] = 1
    return o_h

In [8]:
x = np.random.randint(0,10,10)
x

array([5, 5, 1, 9, 8, 0, 1, 6, 5, 5])

In [9]:
one_hot(x, n=10)

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

In [10]:
a = [0,1,2]

In [12]:
one_hot(a, n=3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])


### Red de clasifiación

Definamos ahora la red que vamos a utilizar para nuestro clasificador. Se compondrá de: una capa de entrada con 4 entradas (ancho y alto de pétalo y sépalo), 5 neuronas en la capa oculta y 3 neuronas en la capa de salida, dado que hay tres clases. *¿Y por qué cinco neuronas en la capa oculta?*

<img src="Figures/rediris.png" width="25%">


### Función de activación softmax

Habíamos dicho que cada neurona tiene como función de activación la función sigmoide. Vamos a cambiar esto para la **última, y solo para la última capa**. En la última capa quitamos la función sigmoide y colocamos la función **softmax**. 
La función softmax es parecida a la sigmoide, ésta también convierte cualquier valor a un valor entre cero y uno. La diferencia está en que la función softmax no trabaja sobre un valor sino sobre un vector. De esta forma, convierte todas las componentes de un vector en valores entre cero y uno, pero, además, garantiza que la suma de todos estos valores sea 1. 
>La salida de la función softmax conforma una **distribución de probabilidad**.

La expresión de la función softmax es:

$$ Softmax(\vec{v})_i = \frac{e^{v_i}}{\sum_{j=1}^{n}{e^{v_j}}} $$

Veamos esto con un poco de código:

In [13]:
# programar función softmax
def softmax(i,v):
    return np.exp(v[i])/np.sum(np.exp(v))

In [14]:
a = np.array([8.34, -2.87, 0.002])


In [15]:
S1 = softmax(0,a)
S1

0.999747275384849

In [16]:
S2 = softmax(1,a)
S2

1.3534714121386372e-05

In [17]:
S3 = softmax(2,a)
S3

0.0002391899010295041

In [18]:
S1+S2+S3

0.9999999999999999

### Función de error

Nuestra función de error va a comparar las salidas de la red con la etiqueta. Supongamos que nuestra red devuelve (salida del softmax) para una determinada entrada los valores: $o_1=0.78, o_2=0.03, o_3=0,19$, y que su correspondiente etiqueta es: $(1,0,0)$ en codificación softmax. Por tanto, el error, para esta muestra, será: 

$$error = (1-0.78)^2 + (0-0.03)^2 + (0-0.19)^2 = 0.0854$$

Pero, dado que tenemos 150 muestras, la función de error final la calcularíamos sumando los 150 errores particulares de cada muestra.



## Creación de Librería de nn

Abrimos un archivo de spyder y trabajaremos en lo siguiente:

- programar las diferentes funciones de activación
- crear la clase Layer
- crear la clase Net




In [20]:
import numpy as np
import pandas as pd

# Creación de la función one-hot
def one_hot(x, n):
    if type(x)==list:
        x = np.array(x)
    x = x.flatten()
    o_h = np.zeros((len(x),n))
    o_h[np.arange(len(x)), x]=1
    return o_h

##----Funciones de activación
# programar función softmax
def softmax(i,v):
    return np.exp(v[i])/np.sum(np.exp(v))

#función sigmoide
def sigmoid(x):
    return 1/(1+np.exp(-x))

#función derivada de sigmoide
def sigmoid_der(x):
    return x*(1-x)

#ReLu
def relu(x):
    if x>=0:
        return x
    else:
        return 0

def relu_der(x):
    if x>=0:
        return 1
    else:
        return 0

def idintity(x):
    return x

def identity_der():
    return 1

#TanH
def tanh(x):
    return 2/(1+np.exp(-2*x)) - 1

def tanh_der(x):
    return 1 - x**2

activ_f = {
    'Sigmoid': (sigmoid, sigmoid_der),
    'ReLu': (relu, relu_der),
    'Identity': (idintity, identity_der),
    'Tanh': (tanh, tanh_der)
}

class Layer:
    """ Layer
    """

    def __init__(self, rows, columns, activ_f=activ_f["Sigmoid"]):
        self.rows = rows
        self.columns = columns + 1  # One column for bias
        self.w = Layer.create_w(self.rows, self.columns)
        self.W_delta = []
        self.b_delta = []
        self.activ_f = activ_f[0]
        self.o_activ_f_derivate = activ_f[1]
        self.error = []
        self.o = None

    @staticmethod
    def create_w(rows, columns):
        return np.random.rand(rows, columns) * 0.1

    def output(self, x):
        x = np.append(x, 1.0)
        print('shape x: ',x.shape)
        print('shape w: ',self.w.shape)
        self.o = self.activ_f(np.dot(self.w, x))
        self.o_derivate = self.o_activ_f_derivate(self.o)
        return self.o

    def __str__(self):
        return "Layer_____________\n     #inputs:" + repr(self.columns-1) + "\n     #neurons: " + repr(self.rows)


class Net:
    """ Net
    """

    def __init__(self, layers):
        self.layers = []
        l_prev = layers[0]
        # conección entre las capas
        for l in layers[1:]:
            self.layers.append(Layer(l, l_prev))
            l_prev = l
        self.number_of_layers = len(self.layers)

    def output(self, x):
        o_aux = x
        for layer in self.layers:
            o_aux = layer.output(o_aux)
        return o_aux
                       
    def train(self, input_data, labels, lr=0.1):
        for x, _y in zip(input_data, labels):
            y = self.output(x)
            error = y - _y
            for index_layer in range(self.number_of_layers)[:0:-1]: 
                error = error * self.layers[index_layer].o_derivate
                self.layers[index_layer].W_delta = np.dot(error.reshape((len(error), 1)),
                                                          (np.append(self.layers[index_layer-1].o, 1.0))
                                                          .reshape(1, len(self.layers[index_layer-1].o)+1))
                error = np.dot(error, self.layers[index_layer].w[:,0:-1])
            error = error*self.layers[0].o_derivate 
            self.layers[0].W_delta = np.dot(error.reshape((len(error),1)), np.append(x,1).reshape((1, len(x)+1)))
            
            for index_layer in range (self.number_of_layers):
                #Actualización de los pesos w = w - lr*w_delta  (w_delta son las derivadas parciales)
                self.layers[index_layer].w -= self.layers[index_layer].W_delta*lr
                


### Mini-batch

La forma de calcular la función de error que acabamos de ver no se hace así en la práctica. Lo entenderemos mejor con la siguiente comparación.

Supongamos que queremos saber la media de altura de todos los habitantes del país. Deberíamos medir a cada persona, sumar todas las alturas y dividir por el número total de personas. Esta es la media exacta, pero nos iba a costar mucho poder llevar a cabo todas estas medidas (somos algo más de 126,2 millones). Otra forma mucho más factible sería tomar una muestra de esta población y calcular la media muestral. Si la muestra es suficientemente variada y amplia obtendremos un valor muy aproximado a la media real pero con mucho menos esfuerzo.

Algo parecido podemos aplicar sobre la función de error. Si, en lugar de utilizar todo el conjunto de datos para calcular el error y luego el gradiente que nos lleve a un nuevo conjunto de pesos y umbrales, tomamos una muestra variada de los datos para calcular el error, el gradiente que obtendremos será muy parecido y nos llevará mucho menos tiempo. Por lo tanto, podremos avanzar mucho más rápido con el gradiente descendente.

A esta muestra (entiéndase como "muestra poblacional", no "muestra" como elemento del *dataset*)  que tomamos del *dataset* lo llamaremos **mini-batch**.

Por ejemplo, en el caso del *dataset* del iris, son 150 muestras, que, una vez bien “barajadas” para que el *mini-batch* sea variado y representativo, podremos tomar mini-lotes de 15 muestras. Teniendo un total de 10 mini-lotes. Con *datasets* pequeños como éste no se aprecia visiblemente la mejora en velocidad, pero con conjuntos de decenas de miles de muestras o más, la reducción de velocidad es drástica.

Por tanto, iremos tomando sucesivos *mini-batchs* de forma iterativa hasta completar el número de estos. Cuando hayamos procesado todos los *mini-batchs* diremos que hemos completado una época (**epoch**), y volveremos a repetir todo el proceso hasta  aproximarnos suficientemente a un mínimo local de la función de error.


### Código

Implementemos ya todos estos conceptos. Para ello, utilizaremos el módulo <code>nn</code> que creamos con anterioridad. Este módulo debe incluir una clase denominada <code>Net</code> para implementar redes sencillas de una forma muy simple.

In [21]:
# cargar dataset de iris
data_iris

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),especie,out
0,5.1,3.5,1.4,0.2,0,0
1,4.9,3.0,1.4,0.2,0,0
2,4.7,3.2,1.3,0.2,0,0
3,4.6,3.1,1.5,0.2,0,0
4,5.0,3.6,1.4,0.2,0,0
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,2
146,6.3,2.5,5.0,1.9,2,2
147,6.5,3.0,5.2,2.0,2,2
148,6.2,3.4,5.4,2.3,2,2


In [22]:
#creacion del minibatch
data_iris = data_iris.values
data_iris

array([[5.1, 3.5, 1.4, 0.2, 0. , 0. ],
       [4.9, 3. , 1.4, 0.2, 0. , 0. ],
       [4.7, 3.2, 1.3, 0.2, 0. , 0. ],
       [4.6, 3.1, 1.5, 0.2, 0. , 0. ],
       [5. , 3.6, 1.4, 0.2, 0. , 0. ],
       [5.4, 3.9, 1.7, 0.4, 0. , 0. ],
       [4.6, 3.4, 1.4, 0.3, 0. , 0. ],
       [5. , 3.4, 1.5, 0.2, 0. , 0. ],
       [4.4, 2.9, 1.4, 0.2, 0. , 0. ],
       [4.9, 3.1, 1.5, 0.1, 0. , 0. ],
       [5.4, 3.7, 1.5, 0.2, 0. , 0. ],
       [4.8, 3.4, 1.6, 0.2, 0. , 0. ],
       [4.8, 3. , 1.4, 0.1, 0. , 0. ],
       [4.3, 3. , 1.1, 0.1, 0. , 0. ],
       [5.8, 4. , 1.2, 0.2, 0. , 0. ],
       [5.7, 4.4, 1.5, 0.4, 0. , 0. ],
       [5.4, 3.9, 1.3, 0.4, 0. , 0. ],
       [5.1, 3.5, 1.4, 0.3, 0. , 0. ],
       [5.7, 3.8, 1.7, 0.3, 0. , 0. ],
       [5.1, 3.8, 1.5, 0.3, 0. , 0. ],
       [5.4, 3.4, 1.7, 0.2, 0. , 0. ],
       [5.1, 3.7, 1.5, 0.4, 0. , 0. ],
       [4.6, 3.6, 1. , 0.2, 0. , 0. ],
       [5.1, 3.3, 1.7, 0.5, 0. , 0. ],
       [4.8, 3.4, 1.9, 0.2, 0. , 0. ],
       [5. , 3. , 1.6, 0.

In [23]:
np.random.shuffle(data_iris)
X_data = data_iris[:,:4].astype(float)
y_data = data_iris[:,-1]

In [25]:
y_data

array([0., 2., 0., 2., 2., 2., 2., 1., 1., 1., 0., 0., 2., 0., 0., 2., 0.,
       0., 2., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 2., 2., 1., 1.,
       2., 2., 1., 0., 0., 1., 2., 2., 0., 0., 0., 0., 1., 1., 2., 2., 2.,
       1., 2., 1., 0., 0., 0., 2., 2., 1., 1., 0., 2., 1., 1., 2., 2., 0.,
       0., 1., 2., 2., 1., 1., 0., 1., 2., 2., 2., 0., 1., 1., 2., 1., 1.,
       0., 2., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1.,
       1., 0., 0., 2., 2., 2., 0., 0., 0., 1., 1., 1., 1., 2., 1., 2., 1.,
       1., 2., 2., 2., 0., 2., 0., 0., 2., 2., 2., 1., 1., 0., 2., 2., 1.,
       1., 0., 0., 0., 2., 0., 2., 1., 2., 1., 2., 2., 0., 2.])

In [26]:
y_data = one_hot(y_data.astype(int), 3)
y_data

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1

In [28]:
batchs_size = 15 #Tamaño del minibartch
epochs = 100  #Número de epocas

#Crear la red de 4 entradas, 5 en una capa oculta, y 3 en la capa de salida
net = Net(layers=[4,5,3])

for _ in range(epochs):
    for i in range(int(data_iris.shape[0]/batchs_size)):
        p = i*batchs_size
        net.train(X_data[p:p+batchs_size], y_data[p:p+batchs_size])

shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
shape x:  (5,)
shape w:  (5, 5)
shape x:

In [29]:
X_data[:15]

array([[4.6, 3.2, 1.4, 0.2],
       [6.7, 3.3, 5.7, 2.1],
       [5.1, 3.8, 1.9, 0.4],
       [5.8, 2.7, 5.1, 1.9],
       [6.8, 3. , 5.5, 2.1],
       [7.7, 2.8, 6.7, 2. ],
       [6. , 3. , 4.8, 1.8],
       [6.7, 3.1, 4.4, 1.4],
       [5.2, 2.7, 3.9, 1.4],
       [5.5, 2.4, 3.7, 1. ],
       [4.8, 3. , 1.4, 0.1],
       [4.7, 3.2, 1.3, 0.2],
       [6.5, 3. , 5.8, 2.2],
       [4.6, 3.6, 1. , 0.2],
       [5.4, 3.4, 1.7, 0.2]])

In [30]:
y_data[:15]

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [31]:
#Predicción 
for x, y in zip(X_data[:15], y_data[:15]):
    y_pred = net.output(x)
    print(y, '---->', np.round(y_pred))

shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
[1. 0. 0.] ----> [1. 0. 0.]
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
[0. 0. 1.] ----> [0. 0. 1.]
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
[1. 0. 0.] ----> [1. 0. 0.]
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
[0. 0. 1.] ----> [0. 0. 1.]
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
[0. 0. 1.] ----> [0. 0. 1.]
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
[0. 0. 1.] ----> [0. 0. 1.]
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
[0. 0. 1.] ----> [0. 0. 1.]
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
[0. 1. 0.] ----> [0. 1. 0.]
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
[0. 1. 0.] ----> [0. 1. 0.]
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
[0. 1. 0.] ----> [0. 1. 0.]
shape x:  (5,)
shape w:  (5, 5)
shape x:  (6,)
shape w:  (3, 6)
[1. 0. 0.] ---->

### Parámetros e hiperparámetros

Es posible que, ya a estas alturas, te estés preguntando cómo se decide el número de capas ocultas que debe tener una red, o cuántas neuronas por capa oculta, cómo establecer el tamaño del mini-batch, el número de épocas... Estos y otros muchos, aún por ver, valores en el diseño de la red se denominan **hiperparámetros**. No son valores que la red aprenda por sí sola, sino que le han de ser dados. No hay un procedimiento sistemático para establecerlos, no hay forma de saber *a priori* cuáles son los mejores valores. En muchos casos, no queda más remedio que probar entre diferentes configuraciones y/o seguir algunas recetas, más o menos fiables, provenientes de la experiencia.

Al conjunto de todos los pesos (y umbrales, en el caso de que los tengamos diferenciados de los pesos) los denominamos **parámetros** y, a diferencia de los hiperparámetros, sí son aprendidos por la red.